Wafer fault detection using FNN.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.base import BaseEstimator, ClassifierMixin
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

# Load the dataset
data = pd.read_csv(r'C:\Users\Shakshi Singh\Documents\ml project\secom.data', delim_whitespace=True, header=None)
labels = pd.read_csv(r'C:\Users\Shakshi Singh\Documents\ml project\secom_labels.data', delim_whitespace=True, header=None)

# Check for null values
print("Null values in dataset:", data.isnull().sum().sum())

# Separate features and target variable
X = data.values
y = labels.iloc[:, 0].replace({-1: 0, 1: 1}).values  # Convert -1 to 0 for binary classification

# Impute missing values (NaN)
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.2, random_state=42)


# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Feature selection using ANOVA F-test
selector = SelectKBest(score_func=f_classif, k=40)  # Select top 40 features
X_train_selected = selector.fit_transform(X_train, y_train)
X_test_selected = selector.transform(X_test)

# Define the model creation function
def create_model():
    model = keras.Sequential([
        layers.Input(shape=(X_train_selected.shape[1],)),
        layers.Dense(256, activation='relu'),  # Increase number of units
        layers.Dropout(0.5),  # Add dropout for regularization
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), 
                  loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Define the custom Keras classifier
class KerasClassifierCustom(BaseEstimator, ClassifierMixin):
    def __init__(self, build_fn=None, epochs=50, batch_size=32, verbose=0):
        self.build_fn = build_fn
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.model = None

    def fit(self, X, y):
        early_stopping = EarlyStopping(monitor='val_loss', patience=10)
        self.model = self.build_fn()
        self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose, validation_split=0.2, callbacks=[early_stopping])

        # self.model.fit(X, y, epochs=self.epochs, batch_size=self.batch_size, verbose=self.verbose)
        return self
        

     

# In your model training

    def predict(self, X):
        return (self.model.predict(X) > 0.5).astype(int)

    def score(self, X, y):
        return self.model.evaluate(X, y, verbose=0)[1]  # Returns accuracy

# Use the custom KerasClassifier
model = KerasClassifierCustom(build_fn=create_model, epochs=50, batch_size=32, verbose=0)
model.fit(X_train_selected, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_selected)

# Evaluate the model
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Now you can use cross_val_score as before
cv_scores = cross_val_score(model, X_train_selected, y_train, cv=10)
print(f'10-fold CV Average Accuracy: {np.mean(cv_scores):.4f}')


C:\Users\Shakshi Singh\AppData\Local\Temp\ipykernel_21468\1989419827.py:16: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(r'C:\Users\Shakshi Singh\Documents\ml project\secom.data', delim_whitespace=True, header=None)
C:\Users\Shakshi Singh\AppData\Local\Temp\ipykernel_21468\1989419827.py:17: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  labels = pd.read_csv(r'C:\Users\Shakshi Singh\Documents\ml project\secom_labels.data', delim_whitespace=True, header=None)
C:\Users\Shakshi Singh\Documents\anaconda\envs\shakshi\Lib\site-packages\sklearn\feature_selection\_univariate_selection.py:112: UserWarning: Features [  5  13  42  49  52  69  97 141 149 178 179 186 189 190 191 192 193 194
 226 229 230 231 232 233 234 235 236 237 240 241 242 243 256 257 258 259
 260 261 262 263 264 265 2

Null values in dataset: 41951
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
Confusion Matrix:
[[289   1]
 [ 24   0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.92      1.00      0.96       290
           1       0.00      0.00      0.00        24

    accuracy                           0.92       314
   macro avg       0.46      0.50      0.48       314
weighted avg       0.85      0.92      0.89       314

10-fold CV Average Accuracy: 0.9258
